In [1]:
from urllib2 import urlopen
from StringIO import StringIO
import gzip
import re

from char_util import ToSimplified

In [2]:
# For downloading and decompressing CCDICT file
with open('cedict_1_0_ts_utf-8_mdbg.txt', 'wb') as f:
    response = urlopen('https://www.mdbg.net/chinese/export/cedict/cedict_1_0_ts_utf-8_mdbg.txt.gz')
    buf = StringIO(response.read())
    txt = gzip.GzipFile(fileobj=buf)
    f.write(txt.read())

In [2]:
def get_cedict_definitions(line): #If I used regex in this function, would it be any slower?
    for i, char in enumerate(line):
        if char == '/':
            begin_index = i + 1
            break
    for i, char in reversed(list(enumerate(line))):
        if char == '/':
            end_index = i
            break
    x = line[begin_index:end_index]
    definitions = x.split('/')
    return definitions

In [3]:
def get_cedict_categ(line):
    definitions = get_cedict_definitions(line)
    try:
        for definition in definitions:
             # This can be expanded to other formats where Buddhism can appear in definitions
            if '(Buddhism)' in definition.split():
                return 'Buddhism'
        # The following must be in order of lowest to highest index required, or else IndexError could be thrown too early
        if definitions[0].split()[1].lower() == 'county':
            return 'county'
        if definitions[0].split()[1].lower() == 'river':
            return 'river'
        if definitions[0].split()[1].lower() == 'lake' or definitions[0].split()[0] == 'Lake':
            return 'lake'
        if definitions[0].split()[2].lower() == 'level' and definitions[0].split()[3].lower() == 'city':
            return 'city'
        if definitions[0].split()[1].lower() == 'district' and definitions[0].split()[4].lower() == 'city':
            return 'district'
    except IndexError as e:
        return 'other'
    return 'other'

In [4]:
def get_pinyin(trad):
    _, _, pinyin = ToSimplified(trad)
    return pinyin

In [5]:
def parse_entry(line):
    tokens = re.split(' ', line)
    entry = {}
    entry["traditional"] = tokens[0]
    entry["simplified"] = tokens[1]
    entry["pinyin"] = get_pinyin(tokens[0])
    return entry

In [ ]:
trad_list = []
temp_dict = {}  # contains mutual bigram info except frequency
mutual_bigram_info = {}
with open('cedict_1_0_ts_utf-8_mdbg.txt', 'rt') as cedict, open('../../hsingyundl/index/ngram_frequencies.txt', 'rt') as bigram_file:
    for line in cedict:
        if line[0] == '#':
            continue
        trad = line.split()[0]
        trad_list.append(trad)
        temp_dict[trad] = parse_entry(line)
    words = set(trad_list) # bigrams is list, words is the same set
    for line in bigram_file:
        info = line.split()
        if info[0] in words:
            entry = temp_dict[info[0]]
            traditional = info[0]
            simplified = entry["simplified"]
            pinyin = entry["pinyin"]
            print("{0}\t{1}\t{2}".format(traditional, simplified, pinyin))


char_util.py:23: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if simplified == trad:
